In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# 새 섹션

#설명




**보건데이터**
* 2012년~2015년 날짜
* 지역(시도명)
* 성별
* 심뇌혈관 질환으로 인한 입원 빈도수(target variable)

**기상데이터**
* 날씨마루 제공
* 기상관측 + 예보 + 기상지수 + 수치모델 : 4가지 기상자료 제공

* **예보 데이터 활용 필수**
* **다양한 기상데이터 활용, 질환 발생 가능성 예측 모델 제시**


최종 검증 데이터 2016.1~2016.12

#0.Setting

In [ ]:
import pandas as pd
import numpy as np                                                                                                                                          
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# ! pip install dataprep
# from dataprep.eda import create_report

In [ ]:
path = "/content/drive/Shareddrives/2022-1 분류 예측 프로젝트/날씨 빅데이터/data/혈관질환"

# back_hospital

In [ ]:
back_hospital = pd.read_csv(path+"/back_hospital.csv", index_col = 0)

In [ ]:
back_hospital

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
1,20120101,강원,1,3
2,20120101,강원,2,3
3,20120101,경기,1,4
4,20120101,경기,2,5
5,20120101,경남,1,2
...,...,...,...,...
49670,20151231,제주,2,0
49671,20151231,충남,1,0
49672,20151231,충남,2,0
49673,20151231,충북,1,1


In [ ]:
back_hospital.columns = [col[int(col.find("."))+1 : ] for  col in back_hospital.columns]

In [ ]:
back_hospital.area.unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [ ]:
len(back_hospital.area.unique())

17

In [ ]:
back_hospital.columns

Index(['yyyymmdd', 'area', 'sex', 'frequency'], dtype='object')

지점코드


2012~2015 : train data set이다.


In [ ]:
STN = pd.read_csv(path+"/염윤석/db_comn_kma_stn.csv", index_col = 0)
STN.columns =[col[int(col.find("."))+1 : ] for  col in STN.columns]
STN = STN[["stn_id", "stn_kor_nm"]]

In [ ]:
len(STN.stn_id)

130

#1.미세먼지 데이터

In [ ]:
pm12 =pd.read_csv(path+"/염윤석/2012_pm10.csv", index_col=0)
pm13 =pd.read_csv(path+"/염윤석/2013_pm10.csv", index_col=0)
pm14 =pd.read_csv(path+"/염윤석/2014_pm10.csv", index_col=0)
pm15 =pd.read_csv(path+"/염윤석/2015_pm10.csv", index_col=0)
pm16 = pd.read_csv(path+"/염윤석/2016_pm10.csv", index_col = 0)

In [ ]:
pm12.columns

Index(['tm', 'stn_id', 'pm10'], dtype='object')

In [ ]:
pm12.stn_id.unique()

array([132, 130, 100, 192, 232, 169, 185, 201, 175, 140, 152,  94, 146,
       108, 156, 160, 136, 102, 135, 121, 229, 143, 456, 119,  90, 115,
       101, 116])

In [ ]:
len(pm12.stn_id.unique())

28

In [ ]:
for s in list(pm12.stn_id.unique()):
  a = np.array(STN.loc[STN.stn_id == s].stn_kor_nm)
  if len(a) != 0:
    b = a[0]
  else: b = "x"
  print("stn_id: ",s, "kor : ", b)

stn_id:  132 kor :  x
stn_id:  130 kor :  울진
stn_id:  100 kor :  대관령
stn_id:  192 kor :  진주
stn_id:  232 kor :  천안
stn_id:  169 kor :  흑산도
stn_id:  185 kor :  고산
stn_id:  201 kor :  강화
stn_id:  175 kor :  진도
stn_id:  140 kor :  군산
stn_id:  152 kor :  울산
stn_id:  94 kor :  광덕산
stn_id:  146 kor :  전주
stn_id:  108 kor :  기상청
stn_id:  156 kor :  광주
stn_id:  160 kor :  구덕산
stn_id:  136 kor :  안동
stn_id:  102 kor :  백령도
stn_id:  135 kor :  추풍령
stn_id:  121 kor :  영월
stn_id:  229 kor :  x
stn_id:  143 kor :  대구
stn_id:  456 kor :  x
stn_id:  119 kor :  수원
stn_id:  90 kor :  속초
stn_id:  115 kor :  울릉도
stn_id:  101 kor :  춘천
stn_id:  116 kor :  관악산


충남 : 132(안면도), 232(천안), 94(광덕산),229(북격렬비도)

경북 : 130(울진), 136(안동), 115(울릉도)

강원 : 100(대관령), 121(영월), 90(속초), 101(춘천)

경남 : 192(진주), 

전남 : 169(흑산도), 175(진도-첨찰산), 

전북 : 185(고산), 140(군산), 146(전주), 

인천 : 201(강화), 102(백령도)

울산 : 152

서울 : 108(기상청), 116(관악산)

광주 : 156

부산 : 160(구덕산)

충북 : 135(추풍령), 

대구 : 143

경기 : 119(수원), 456(연천)

대전,세종 == 충남


In [ ]:
set(pm12.stn_id.unique()) == set(pm15.stn_id.unique())

False

In [ ]:
set(pm15.stn_id.unique()) - set(pm12.stn_id.unique())

{268, 273, 286, 430, 432, 501}

In [ ]:
set(pm12.stn_id.unique()) - set(pm15.stn_id.unique())

{456}

In [ ]:
##15에만 있는 미세먼지 데이터가 있다. -
for s in list(set(pm15.stn_id.unique()) - set(pm12.stn_id.unique())):
  a = np.array(STN.loc[STN.stn_id == s].stn_kor_nm)
  if len(a) != 0:
    b = a[0]
  else: b = "x"
  print("stn_id: ",s, "kor : ", b)

stn_id:  268 kor :  x
stn_id:  430 kor :  x
stn_id:  432 kor :  x
stn_id:  273 kor :  문경
stn_id:  501 kor :  x
stn_id:  286 kor :  x


268 : 

어차피 필요 없다.

지점 번호가 년도별로 다 같다.

지점 번호를 back_hospital의 "지역"에 맞게 전처리


지역

'강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산','인천', '전남', '전북', '제주', '충남', '충북'

각 지역번호에 맞는 지역을 찾는다.

이후 지역코드를 grouping을 한다.

**tm을 yyyymmdd로 변환**

*   지점별, 날짜별 pm10 평균치 구하기
*   각 지점의 지역명 달기



In [ ]:
region = {}
region["충남"]= [132, 232, 94, 229, 129, 177, 238, 236, 235]
region["충북"] = [135, 127, 226, 221, 131]
region["경북"] = [130,136,115, 271, 138, 137, 283, 281, 279, 278, 277, 276, 273, 272]
region["경남"] = [192, 263, 257, 255, 253, 162, 155, 295, 294, 289, 288, 285, 284, 264]
region["강원"] = [100,121,90,101, 93, 216, 217, 214, 212, 211, 114, 106, 105, 104, 95]
region["전남"] =[169,175, 260, 256, 168, 259, 258, 252, 174, 170,165, 164, 268, 261, 262]
region["전북"] = [185,140,146, 251, 254, 248, 247, 245, 244, 243, 172]
region["인천"] = [201,102,112]
region["울산"] = [152]
region["서울"] = [108,116]
region["광주"]= [156]
region["부산"] = [160, 159]
region["대구"] = [143]
region["경기"] = [119,456, 98, 203, 99]
region["대전"] = [133]
region["세종"] =[239]
region["제주"] = [265, 184, 185, 187, 188, 189]

In [ ]:
def mapping_region(x,region):
  for key in region.keys():
    if x in region[key]:
      return key
      break
    else:
      continue

def preprocessing1(pm,region):
  pm["yyyymmdd"] = pm.tm.map(lambda x : x[:4]+x[5:7]+x[8:10])
  pm = pm.astype({"yyyymmdd" : "int"}) ##string을 index한거기에 int로 바꿔야한다.
  pm["hhnn"] = pm.tm.map(lambda x: x[-10:])
  pm.drop(["tm"],axis = 1, inplace = True)

  temp = pm.groupby(["stn_id", "yyyymmdd"], as_index = False).mean()

  temp["area"] = temp.stn_id.map(lambda x : mapping_region(x,region))
  
  return temp

In [ ]:
pm_12 = preprocessing1(pm12,region)
pm_13 = preprocessing1(pm13,region)
pm_14 = preprocessing1(pm14,region)
pm_15 = preprocessing1(pm15,region)
pm_16 = preprocessing1(pm16,region)

In [ ]:
# 다합치자
pm = pd.concat([pm_12,pm_13,pm_14,pm_15,pm_16], axis =0)
pm.to_csv(path+"/염윤석/pm_10")

In [ ]:
pm.loc[pm.isna().any(axis = 1)]

,stn_id,yyyymmdd,pm10,area
9362,286,20151015,48.400000,None
9363,430,20150501,29.571429,None
9364,432,20151029,10.660714,None
9365,432,20151030,14.520833,None
9366,432,20151031,13.538194,None
...,...,...,...,...
10563,501,20161227,27.468750,None
10564,501,20161228,18.013937,None
10565,501,20161229,29.097222,None
10566,501,20161230,27.899306,None


back_hospital에 데이터 넣기

In [ ]:
back_hospital["pm_10"] = 0
back_hospital

,yyyymmdd,area,sex,frequency,pm_10
1,20120101,강원,1,3,0
2,20120101,강원,2,3,0
3,20120101,경기,1,4,0
4,20120101,경기,2,5,0
5,20120101,경남,1,2,0
...,...,...,...,...,...
49670,20151231,제주,2,0,0
49671,20151231,충남,1,0,0
49672,20151231,충남,2,0,0
49673,20151231,충북,1,1,0


In [ ]:
for day in list(back_hospital.yyyymmdd.unique()):
  for area in list(back_hospital.area.unique()):
    a = back_hospital.loc[(back_hospital.yyyymmdd == day) & (back_hospital.area == area)]
    b = pm.loc[(pm.yyyymmdd == day) & (pm.area == area)]

    if len(b)>0: ## 찾은게 있다면
      pm_value = b.pm10.mean()
    else:## 찾은 게 없다면
      pm_value = 0

    #특정 값 바꿀려면 loc[index, column]을 써야한다. index는 조건문도 가능
    back_hospital.loc[(back_hospital.yyyymmdd == day) & (back_hospital.area == area),"pm_10"] = pm_value

In [ ]:
back_hospital

,yyyymmdd,area,sex,frequency,pm_10
1,20120101,강원,1,3,63.872640
2,20120101,강원,2,3,63.872640
3,20120101,경기,1,4,97.440829
4,20120101,경기,2,5,97.440829
5,20120101,경남,1,2,35.337979
...,...,...,...,...,...
49670,20151231,제주,2,0,0.000000
49671,20151231,충남,1,0,44.624107
49672,20151231,충남,2,0,44.624107
49673,20151231,충북,1,1,46.167273


In [ ]:
back_hospital.to_csv(path+"/염윤석/back_with_pm10.csv")

In [ ]:
back_hospital.loc[back_hospital.pm_10 ==0]

,yyyymmdd,area,sex,frequency,pm_10
13,20120101,대전,1,4,0.0
14,20120101,대전,2,1,0.0
19,20120101,세종,1,0,0.0
20,20120101,세종,2,0,0.0
29,20120101,제주,1,0,0.0
...,...,...,...,...,...
49654,20151231,대전,2,0,0.0
49659,20151231,세종,1,0,0.0
49660,20151231,세종,2,0,0.0
49669,20151231,제주,1,0,0.0


In [ ]:
back_hospital.loc[back_hospital.pm_10 ==0].area.unique()

array(['대전', '세종', '제주', '부산', '경남', '울산', '대구', '충북', '광주', '경기', '경북',
       '서울', '인천', '전남', '전북', '충남', '강원'], dtype=object)

해당 지역의 해당 날짜에 해당하는 미세먼지 데이터가 없는 경우, 결측치로 처리된다.

##1-1 test : pm_10(2016년도) 전처리

In [ ]:
test_16 =pd.read_csv(path + "/2-2_검증데이터셋.csv",encoding = "cp949")
test_16.yyyymmdd = test_16.yyyymmdd.map(lambda x : x[:4]+x[5:7]+x[8:])
test_16 = test_16.astype({"yyyymmdd" : "int"})
test_16["pm_10"] = 0

In [ ]:
for day in list(test_16.yyyymmdd.unique()):
  for area in list(test_16.area.unique()):
    a = test_16.loc[(test_16.yyyymmdd == day) & (test_16.area == area)]
    b = pm.loc[(pm.yyyymmdd == day) & (pm.area == area)]

    if len(b)>0: ## 찾은게 있다면
      pm_value = b.pm10.mean()
    else:## 찾은 게 없다면
      pm_value = 0

    #특정 값 바꿀려면 loc[index, column]을 써야한다. index는 조건문도 가능
    test_16.loc[(test_16.yyyymmdd == day) & (test_16.area == area),"pm_10"] = pm_value

In [ ]:
test_16.to_csv(path+"/염윤석/test_with_pm10.csv")
test_16

,yyyymmdd,area,sex,frequency,pm_10
0,20160101,경기,1,NaN,61.181818
1,20160102,경기,1,NaN,44.652778
2,20160103,경기,1,NaN,79.170139
3,20160104,경기,1,NaN,61.024306
4,20160105,경기,1,NaN,28.343750
...,...,...,...,...,...
12405,20161227,제주,2,NaN,0.000000
12406,20161228,제주,2,NaN,0.000000
12407,20161229,제주,2,NaN,0.000000
12408,20161230,제주,2,NaN,0.000000


In [ ]:
test_16.loc[test_16.pm_10 ==0]

,yyyymmdd,area,sex,frequency,pm_10
2421,20160820,충북,1,NaN,0.0
2422,20160821,충북,1,NaN,0.0
2786,20160820,충북,2,NaN,0.0
2787,20160821,충북,2,NaN,0.0
5473,20161230,대구,1,NaN,0.0
...,...,...,...,...,...
12405,20161227,제주,2,NaN,0.0
12406,20161228,제주,2,NaN,0.0
12407,20161229,제주,2,NaN,0.0
12408,20161230,제주,2,NaN,0.0


In [ ]:
test_16.loc[test_16.pm_10 ==0].area.unique()

array(['충북', '대구', '세종', '대전', '부산', '제주'], dtype=object)

#2. 전처리 완료된 데이터로 모델링

In [ ]:
data = pd.read_csv(path+"/염윤석/back_hospital_new.csv",index_col = 0)
data

,yyyymmdd,area,sex,frequency,min_temp,max_temp,avg_air_moisture,avg_air_pressure,solar_hour,temp_change,HBP,DB,Smoke,pm_10,factory,ingu,D01,D02
0,20120101,강원,1,3,-7.176923,1.238462,65.307692,997.692308,9.623077,8.415385,20.66,7.64,46.39,63.872640,1.9,772718.0,1,1
1,20120101,강원,2,3,-7.176923,1.238462,65.307692,997.692308,9.623077,8.415385,20.66,7.64,2.65,63.872640,1.9,762617.0,1,1
2,20120101,경기,1,4,-7.266667,0.933333,68.666667,1017.900000,9.666667,8.200000,19.62,7.64,44.43,97.440829,35.7,6020446.0,1,1
3,20120101,경기,2,5,-7.266667,0.933333,68.666667,1017.900000,9.666667,8.200000,19.62,7.64,3.70,97.440829,35.7,5928150.0,1,1
4,20120101,경남,1,2,-2.380000,7.160000,57.800000,1017.640000,9.840000,9.540000,13.54,4.98,35.71,35.337979,10.1,1665308.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,20151231,제주,2,0,5.700000,10.833333,63.033333,1026.466667,10.000000,5.133333,20.01,6.73,2.58,0.000000,0.4,310967.0,3,3
49670,20151231,충남,1,0,-2.500000,6.225000,85.450000,1028.800000,9.700000,8.725000,17.31,6.97,32.07,44.624107,4.9,1054439.0,2,2
49671,20151231,충남,2,0,-2.500000,6.225000,85.450000,1028.800000,9.700000,8.725000,17.31,6.97,2.77,44.624107,4.9,1023210.0,2,2
49672,20151231,충북,1,1,-3.050000,4.100000,77.850000,1012.275000,9.700000,7.150000,13.63,5.50,26.22,46.167273,4.1,798896.0,2,3


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49674 entries, 0 to 49673
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   yyyymmdd          49674 non-null  int64  
 1   area              49674 non-null  object 
 2   sex               49674 non-null  int64  
 3   frequency         49674 non-null  int64  
 4   min_temp          49674 non-null  float64
 5   max_temp          49674 non-null  float64
 6   avg_air_moisture  49674 non-null  float64
 7   avg_air_pressure  49674 non-null  float64
 8   solar_hour        49674 non-null  float64
 9   temp_change       49674 non-null  float64
 10  HBP               49674 non-null  float64
 11  DB                49674 non-null  float64
 12  Smoke             49674 non-null  float64
 13  pm_10             49674 non-null  float64
 14  factory           49674 non-null  float64
 15  ingu              49674 non-null  float64
 16  D01               49674 non-null  int64 

In [ ]:
data.D01.min()

1

categorical :area

numerical : rest of all

## 전처리

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


def dummy_sex(x):
  if x == 2:
    return 0
  else:
    return x


num_cols = ['min_temp', 'max_temp', 'avg_air_moisture', 'avg_air_pressure',
       'solar_hour', 'temp_change', 'HBP', 'DB', 'Smoke', 'pm_10', 'factory',
       'ingu', 'D01', 'D02']

area와 frequency를 뺀 데이터로 x_train 데이터로 삼았다.(데이터 내용이 지역의 특성을 반영한다고 판단)

나중에 area도 포함시켜서 시도해봐도 좋을 것 같다.

In [ ]:
X_train = data.drop(["yyyymmdd","area","frequency"],axis = 1)
y_train = data.frequency

In [ ]:
# 성별은 categorical로 인코딩
#2만 바꿔준다.


X_train.sex = X_train.sex.map(lambda x: dummy_sex(x))

In [ ]:
X_train.columns

Index(['sex', 'min_temp', 'max_temp', 'avg_air_moisture', 'avg_air_pressure',
       'solar_hour', 'temp_change', 'HBP', 'DB', 'Smoke', 'pm_10', 'factory',
       'ingu', 'D01', 'D02'],
      dtype='object')

In [ ]:
#scaling 

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])


같은 방식으로 test set 전처리

In [ ]:
test = pd.read_csv(path+"/염윤석/test.csv",index_col = 0)
test

,yyyymmdd,area,sex,frequency,min_temp,max_temp,avg_air_moisture,avg_air_pressure,solar_hour,temp_change,HBP,DB,Smoke,pm_10,factory,ingu,D01,D02
0,20160101,경기,1,NaN,-5.533333,3.733333,82.700000,1024.700000,9.666667,9.266667,22.16,8.93,39.76,61.181818,35.6,6306634,3,3
1,20160102,경기,1,NaN,-0.100000,9.766667,80.033333,1016.666667,9.666667,9.866667,22.16,8.93,39.76,44.652778,35.6,6306634,2,2
2,20160103,경기,1,NaN,0.200000,9.766667,87.666667,1013.200000,9.700000,9.566667,20.45,8.09,39.76,79.170139,35.6,6306634,2,2
3,20160104,경기,1,NaN,-1.066667,7.333333,50.433333,1016.900000,9.700000,8.400000,20.45,8.09,39.76,61.024306,35.6,6306634,2,2
4,20160105,경기,1,NaN,-4.033333,2.566667,36.133333,1020.866667,9.700000,6.600000,14.81,5.81,39.76,28.343750,35.6,6306634,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12405,20161227,제주,2,NaN,3.100000,9.575000,63.600000,1018.100000,9.975000,6.475000,19.56,6.67,2.58,0.000000,0.4,319052,3,3
12406,20161228,제주,2,NaN,2.800000,7.975000,59.000000,1023.025000,10.000000,5.175000,18.94,7.73,2.58,0.000000,0.4,319052,3,3
12407,20161229,제주,2,NaN,3.675000,8.000000,64.425000,1024.475000,10.000000,4.325000,18.94,7.73,2.58,0.000000,0.4,319052,3,3
12408,20161230,제주,2,NaN,2.550000,8.150000,63.900000,1026.350000,10.000000,5.600000,20.63,8.61,2.58,0.000000,0.4,319052,3,3


In [ ]:
X_test = test.drop(["yyyymmdd","area","frequency"],axis = 1)
X_test.sex = X_test.sex.map(lambda x: dummy_sex(x))
X_test[num_cols] = scaler.transform(X_test[num_cols])


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error 
import xgboost as xgb
import catboost as ctb

In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.23.2


!pip install pycaret[full]

!pip install xgboost
!pip install --upgrade xgboost
!pip install catboost

!pip install markupsafe==2.0.1

!pip install Jinja2


In [ ]:
from pycaret.utils import enable_colab
enable_colab()

from pycaret.regression import *

Colab mode enabled.


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
sub = pd.read_csv(path + "/2-2_검증데이터셋.csv", encoding = "cp949")

##Lasso

In [ ]:
lasso = Lasso(fit_intercept = True, random_state = 42)

parmas = {"alpha" : [0.1, 0.2, 0.4, 0.6],
          "max_iter": [500, 1000, 2000],
          "tol": [0.0001, 0.00001, 0.001, 0.01]}
grid_lasso = GridSearchCV(lasso, param_grid = parmas, n_jobs = -1, cv = 5, scoring = "neg_mean_squared_error")
grid_lasso.fit(X_train, y_train)

lasso_best = grid_lasso.best_estimator_
print("Best : ", lasso_best)
pred_train = lasso_best.predict(X_train)
print("rmse : ", np.sqrt(mean_squared_error(y_train, pred_train)))
lasso_pred = lasso_best.predict(X_test)

Best :  Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=500, normalize=False,
      positive=False, precompute=False, random_state=42, selection='cyclic',
      tol=1e-05, warm_start=False)
rmse :  1.291598990094273


In [ ]:
result_lasso = sub.copy()
result_lasso.frequency = lasso_pred
result_lasso = result_lasso[["yyyymmdd", "area", "sex", "frequency"]]
result_lasso.frequency = result_lasso.frequency.map(lambda x : round(x))
result_lasso.to_csv(path+ "/염윤석/result/lasso.csv", index = False, encoding = "cp949")

##Ridge

In [ ]:
ridge = Ridge(fit_intercept = True, random_state = 42)

parmas = {"alpha" : [0.1, 0.2, 0.4, 0.6],
          "max_iter": [500, 1000, 2000],
          "tol": [0.0001, 0.00001, 0.001, 0.01]}
grid_ridge = GridSearchCV(ridge, param_grid = parmas, n_jobs = -1, cv = 5, scoring = "neg_mean_squared_error")
grid_ridge.fit(X_train, y_train)

ridge_best = grid_ridge.best_estimator_
print("Best : ", ridge_best)
pred_train = ridge_best.predict(X_train)
print("rmse : ", np.sqrt(mean_squared_error(y_train, pred_train)))
ridge_pred = ridge_best.predict(X_test)

Best :  Ridge(alpha=0.6, copy_X=True, fit_intercept=True, max_iter=500, normalize=False,
      random_state=42, solver='auto', tol=0.0001)
rmse :  1.2780780583602562


In [ ]:
result_ridge = sub.copy()
result_ridge.frequency = ridge_pred
result_ridge = result_ridge[["yyyymmdd", "area", "sex", "frequency"]]
result_ridge.frequency = result_ridge.frequency.map(lambda x : round(x))
result_ridge.to_csv(path+ "/염윤석/result/ridge.csv", index = False, encoding = "cp949")

##Elastic Net

In [ ]:
elastic = ElasticNet(fit_intercept = True, random_state = 42)

parmas = {"alpha" : [0.1, 0.2, 0.4, 0.6],
          "l1_ratio": [0.1, 0.2, 0.5, 0.7, 0.8, 0.9],
          "max_iter": [500, 1000, 2000],
          "tol": [0.0001, 0.00001, 0.001, 0.01]}
grid_elastic = GridSearchCV(elastic, param_grid = parmas, n_jobs = -1, cv = 5, scoring = "neg_mean_squared_error")
grid_elastic.fit(X_train, y_train)

elastic_best = grid_elastic.best_estimator_
print("Best : ", elastic_best)
pred_train = elastic_best.predict(X_train)
print("rmse : ", np.sqrt(mean_squared_error(y_train, pred_train)))
elastic_pred = elastic_best.predict(X_test)

Best :  ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.1,
           max_iter=500, normalize=False, positive=False, precompute=False,
           random_state=42, selection='cyclic', tol=1e-05, warm_start=False)
rmse :  1.2834343121675946


In [ ]:
result_elastic = sub.copy()
result_elastic.frequency = elastic_pred
result_elastic = result_elastic[["yyyymmdd", "area", "sex", "frequency"]]
result_elastic.frequency = result_elastic.frequency.map(lambda x : round(x))
result_elastic.to_csv(path+ "/염윤석/result/elastic.csv", index = False, encoding = "cp949")

## Extratree

In [ ]:
extra = ExtraTreesRegressor(random_state = 42)

parmas = {"n_estimators" : [50,100, 150,200],
        "max_depth" : [5,6,8,10, 20],
        "max_leaf_nodes" : [5, 10, 20, 40, 60, 80],
        }
grid_extra = GridSearchCV(extra, param_grid = parmas, n_jobs = -1, cv = 5, scoring = "neg_mean_squared_error")
grid_extra.fit(X_train, y_train)

extra_best = grid_extra.best_estimator_
print("Best : ", extra_best)
print("rmse : ", extra_best.score(X_train, y_train))
extra_pred = extra_best.predict(X_test)

KeyboardInterrupt: ignored

In [ ]:
result_extra = sub.copy()
result_extra.frequency = extra_pred
result_extra = result_extra[["yyyymmdd", "area", "sex", "frequency"]]
result_extra.frequency = result_extra.frequency.map(lambda x : round(x))
result_extra.to_csv(path+ "/염윤석/result/extra.csv", index = False, encoding = "cp949")

##AutoML

In [ ]:
df_train = X_train.copy()
df_train["frequency"] = y_train

In [ ]:
def automl_model(df_train):

    reg = setup(data = df_train, target = "frequency", train_size = 0.8, fold = 5)
    top_5 = compare_models(sort = "mse", n_select = 5)
    tuned_top5 = [tune_model(i,n_iter = 10) for i in top_5]
    blend5_stack = stack_models(estimator_list=tuned_top5)
    final_model = finalize_model(blend5_stack)

    return final_model, tuned_top5

In [ ]:

final_model, tuned_top5 = automl_model(df_train)


path = "/content/drive/Shareddrives/2022-1 분류 예측 프로젝트/날씨 빅데이터/data/혈관질환"
with open(path + "/염윤석/result/automl.pkl", "wb") as f:
  pickle.dump(tuned_top5, f)

pred = predict_model(final_model, data = X_test)
automl_pred = pred.Label


result_automl = sub.copy()
result_automl.frequency = automl_pred
result_automl = result_automl[["yyyymmdd", "area", "sex", "frequency"]]
result_automl.frequency = result_automl.frequency.map(lambda x : round(x))
result_automl.to_csv(path+ "/염윤석/result/automl.csv", index = False, encoding = "cp949")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8994,1.5247,1.2348,0.4542,0.4879,0.4361
1,0.9325,1.6315,1.2773,0.4211,0.4972,0.4588
2,0.9136,1.5761,1.2554,0.4508,0.4901,0.4468
3,0.9178,1.6074,1.2678,0.4247,0.4961,0.4504
4,0.9111,1.6053,1.2670,0.4340,0.4880,0.4330
Mean,0.9149,1.5890,1.2605,0.4370,0.4919,0.4450
Std,0.0107,0.0366,0.0146,0.0134,0.0040,0.0095


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

In [ ]:
pred_train = predict_model(final_model, data = X_train)
result_train = pred_train.Label

print("rmse:", np.sqrt(mean_squared_error(y_train,result_train)))

rmse: 1.2387845684580434


In [ ]:
with open(path + '/염윤석/result/automl.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=4,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.001, min_impurity_split=None,
                           min_samples_leaf=2, min_samples_split=4,
                           min_weight_fraction_leaf=0.0, n_estimators=180,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2280, subsample=0.35, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False),
 LGBMRegressor(bagging_fraction=1.0, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.3, max_depth=-1,
               min_child_samples=6, min_child_weight=0.001, min_split_gain=0.6,
             

##DNN(Deep Neural Network)

https://ysyblog.tistory.com/101

https://zsunn.tistory.com/entry/AI-DNNDeep-Neural-Network%EC%9D%98-%EC%9D%B4%ED%95%B4-%EB%B0%8F-%EC%8B%A4%EC%8A%B5

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

np.random.seed(1)
tf.random.set_seed(1)

#hyperparameter
learning_rate = 0.001
n_epochs = 5 #에폭수 : 전체 데이터가 지나가는 수
n_batchs = 500

## Fold Ensemble

# new things - 예보데이터

윤병 : 최고기온, 최저기온, 습도 완료

년도별, 지역별, 지역 내 소지역으로 나눠져있음

